<p style="font-weight:bold;"> <span style="font-size: 36px"> Ifrs Variable </span> </p>

In [0]:
//#!eval-notebook "Data/InitSystemorphToMemoryForTesting"

# Set Up

In [0]:
var pathToBm = "./Data/IfrsVariableBenchmarks/";

In [0]:
public record BenchmarkMetadata(string FileName, string ReportingNode, int Year, int Month, string Scenario = null){} 

In [0]:
var excludedDataNodes = new string[]{};
var includedDataNodes = new string[]{};

In [0]:
var numberOfProjections = (await DataSource.Query<ProjectionConfiguration>().ToArrayAsync()).Count();

# Functions

In [0]:
public bool IsCorrectValue(double value) => Math.Abs(value) <= BenchmarkPrecision;

In [0]:
public static NumberStyles AllowedNumberStyles = NumberStyles.Float | NumberStyles.AllowLeadingWhite | NumberStyles.AllowTrailingWhite | NumberStyles.AllowLeadingSign | NumberStyles.AllowTrailingSign | NumberStyles.AllowDecimalPoint;

In [0]:
public double ParseIfrsVariableValue(IDataRow row, string nameOfIfrsVariableValue)
{
    return (row.Table.Columns.Contains(nameOfIfrsVariableValue)
                        ? (double.TryParse(row[nameOfIfrsVariableValue]?.ToString(), AllowedNumberStyles, CultureInfo.InvariantCulture, out var value) 
                                ? value 
                                : throw new Exception(nameOfIfrsVariableValue + " cannot be parsed."))
                        : default(double));
}

In [0]:
public IfrsVariable CreateVariable(IDataRow row, Guid partition) {
    var values = Enumerable.Range(0, numberOfProjections).Select(x => ParseIfrsVariableValue(row, "Values" + x.ToString())).ToArray();
    return new IfrsVariable(){AocType = row.Field<string>(nameof(IfrsVariable.AocType)),
                          Partition = partition,
                          Novelty = row.Field<string>(nameof(IfrsVariable.Novelty)),
                          DataNode = row.Field<string>(nameof(IfrsVariable.DataNode)),
                          EstimateType = row.Field<string>(nameof(IfrsVariable.EstimateType)),
                          AmountType = row.Field<string>(nameof(IfrsVariable.AmountType)),
                          AccidentYear = Int32.TryParse((row.Field<string>(nameof(IfrsVariable.AccidentYear))), out var accidentYear) ? accidentYear : (int?)null,
                          EconomicBasis = row.Field<string>(nameof(IfrsVariable.EconomicBasis)),
                          Values = values.Prune().Count() == 0 ? new []{0d} : values.Prune(),
    };
}    

In [0]:
public class BenchmarkTestResult
{
    public string ErrorMessage {get; set;}
    public double[] ExpectedValue {get; set;}
    public double[] ComputedValue {get; set;}

    public BenchmarkTestResult (string message)
    {
        ErrorMessage = message;
    }

    public BenchmarkTestResult(string message, double[] expValue, double[] compValue){
        ErrorMessage = message;
        ExpectedValue = expValue;
        ComputedValue = compValue;
    }
}

In [0]:
var includingProperties = typeof(IfrsVariable).GetIdentityProperties().Select(x=>x.Name).ToArray();

In [0]:
public List<BenchmarkTestResult> CompareAgainstBm(List<BenchmarkTestResult> errors, IEnumerable<IfrsVariable> expected, IEnumerable<IfrsVariable> computed){
    var comparer = IfrsVariableComparer.Instance(true); 
    var comparerIfrs =  IfrsVariableComparer.Instance(false, BenchmarkPrecision);
    var expectedNotComputed = expected.Where(x => x.Values.Any(y => Math.Abs(y) > BenchmarkPrecision)).Except(computed, comparer);
    var computedNotExpected = computed.Where(x => x.Values.Any(y => Math.Abs(y) > BenchmarkPrecision)).Except(expected, comparer);
    if (expectedNotComputed.Any()){
        foreach(var element in expectedNotComputed){
            errors.Add(new BenchmarkTestResult("Extra expected variable for: Partition " + element.Partition + ", " + element.ToIdentityString(), element.Values, null));
        }
    }
    if (computedNotExpected.Any()){
        foreach(var element in computedNotExpected){
            errors.Add(new BenchmarkTestResult("Extra computed variable for: Partition " + element.Partition  + ", " + element.ToIdentityString(), null, element.Values));
        }
    }
    foreach(var benchmarkVariable in expected){
        var computedVariable = computed.FirstOrDefault(x => comparer.Equals(benchmarkVariable, x)) ?? null;
        if (computedVariable == null) continue;
        if (!comparerIfrs.Equals(benchmarkVariable, computedVariable)) errors.Add(new BenchmarkTestResult("Values do not match. Partition " + benchmarkVariable.Partition + ", " + benchmarkVariable.ToIdentityString(), 
                                                                                                            benchmarkVariable.Values, computedVariable.Values)); 
    }
    return errors;
}

# Test Runner

In [0]:
public bool ValidateMainTab(IDataRow mainRow, string reportingNode, int year, int month)
{
    if (!int.TryParse(mainRow[nameof(PartitionByReportingNodeAndPeriod.Year)]?.ToString(), NumberStyles.AllowLeadingWhite | NumberStyles.AllowTrailingWhite , CultureInfo.InvariantCulture, out var mainYear))
            throw new Exception("Value Year cannot be parsed."); 
    if (!int.TryParse(mainRow[nameof(PartitionByReportingNodeAndPeriod.Month)]?.ToString(), NumberStyles.AllowLeadingWhite | NumberStyles.AllowTrailingWhite , CultureInfo.InvariantCulture, out var mainMonth))
            throw new Exception("Value Month cannot be parsed."); 
    var mainReportingNode = mainRow[nameof(PartitionByReportingNodeAndPeriod.ReportingNode)].ToString();
    
    return reportingNode == mainReportingNode && year == mainYear && month == mainMonth;
}

In [0]:
public async Task<List<BenchmarkTestResult>> CheckIfrsVariablesValuesAsync(BenchmarkMetadata[] bmFiles, string[] excludedDataNodes, string[] includedDataNodes)
{
    var errorList = new List<BenchmarkTestResult>();
    foreach (var g in bmFiles.GroupBy(x => (ReportingNode: x.ReportingNode, Year: x.Year, Month: x.Month, Scenario : x.Scenario)))
    {
        var reportingNode = g.Key.ReportingNode;
        var year = g.Key.Year;
        var month = g.Key.Month;
        var scenario = g.Key.Scenario;
    
        await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(new {ReportingNode = reportingNode, Year = year, Month = month, Scenario = scenario});
        var partition = (await DataSource.Query<PartitionByReportingNodeAndPeriod>().ToArrayAsync())
                            .SingleOrDefault(x => x.ReportingNode == reportingNode && x.Year == year && x.Month == month && x.Scenario == scenario)?.Id ?? 
                            throw new Exception("Partition not found.");
    
        var computedVariablesByEstimateType = (await DataSource.Query<IfrsVariable>()
                                                            .Where(v => excludedDataNodes != null 
                                                                            ? !excludedDataNodes.Contains(v.DataNode) 
                                                                            : true)
                                                            .Where(v => includedDataNodes != null 
                                                                            ? includedDataNodes.Contains(v.DataNode) 
                                                                            : true)
                                                            .ToArrayAsync())
                                                            .ToDictionaryGrouped(x => x.EstimateType, x => x.ToArray());

        foreach (var bmfile in g)
        {
            var extension = System.IO.Path.GetExtension(bmfile.FileName);
            var stream = await Project.FileStorage.ReadAsync(bmfile.FileName);
            //Read Bm csv
            var bmDataTable = (await DataSetReader.ReadFromStream(stream).WithContentType(extension).ExecuteAsync()).DataSet;

            var mainTab = bmDataTable.Tables[Main].Rows.First();
            if(!ValidateMainTab(mainTab, reportingNode, year, month))
                throw new Exception("Metadata and main tab of the Benchmark are different.");

            var bmTableName = bmDataTable.Tables.Single(x => x.TableName != Main).TableName;
            var bmVariables = bmDataTable.Tables[bmTableName].Rows.Select(x => CreateVariable(x, partition));

            //Get the computed Data
            var computedVariables = computedVariablesByEstimateType.TryGetValue(bmTableName, out var ret) ? ret : Enumerable.Empty<IfrsVariable>();

            //CompareAgainstBm
            if(!computedVariables.Any() && bmVariables.Any())
                errorList.Add(new BenchmarkTestResult("No variables are computed for: "+reportingNode+year+month+scenario+" and EstimateType: " + bmTableName));
            if(computedVariables.Any() && !bmVariables.Any())
                errorList.Add(new BenchmarkTestResult("Empty benchmark provided for: "+reportingNode+year+month+scenario+" and EstimateType: " + bmTableName));
            if(computedVariables.Any() && bmVariables.Any())
                errorList = errorList.Concat(CompareAgainstBm(errorList, bmVariables, computedVariables)).Distinct().ToList();
        }
    }
    return errorList;
}

## Projection 0 Test

### BenchMark

In [0]:
var bmFiles = new BenchmarkMetadata[]{
    //2020 Q4
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12__A.csv"   , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12__AA.csv"  , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12__APA.csv" , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12__BE.csv"  , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12__BEPA.csv", "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12__C.csv"   , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12__DA.csv"  , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12__F.csv"   , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12__OA.csv"  , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12__RA.csv"  , "CH", 2020, 12),
    // 2020 Q4 MTUP10pct
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_MTUP10pct_BE.csv"   , "CH", 2020, 12, "MTUP10pct"),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_MTUP10pct_BEPA.csv" , "CH", 2020, 12, "MTUP10pct"),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_MTUP10pct_A.csv"    , "CH", 2020, 12, "MTUP10pct"),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_MTUP10pct_APA.csv"  , "CH", 2020, 12, "MTUP10pct"),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_MTUP10pct_DA.csv"   , "CH", 2020, 12, "MTUP10pct"),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_MTUP10pct_C.csv"    , "CH", 2020, 12, "MTUP10pct"),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_MTUP10pct_L.csv"    , "CH", 2020, 12, "MTUP10pct"),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_MTUP10pct_LR.csv"   , "CH", 2020, 12, "MTUP10pct"),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_MTUP10pct_F.csv"    , "CH", 2020, 12, "MTUP10pct"),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_MTUP10pct_RA.csv"   , "CH", 2020, 12, "MTUP10pct"),
    // 2021 Q1
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__A.csv"    , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__AA.csv"   , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__APA.csv"  , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__BE.csv"   , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__BEPA.csv" , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__C.csv"    , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__DA.csv"   , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__F.csv"    , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__L.csv"    , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__LR.csv"   , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__OA.csv"   , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__RA.csv"   , "CH", 2021, 3),
};

### Test

In [0]:
excludedDataNodes = new string[]{"DTP1.1"};
includedDataNodes = null;
var errorList = await CheckIfrsVariablesValuesAsync(bmFiles, excludedDataNodes, includedDataNodes);

In [0]:
errorList.Count().Should().Be(0);

## Projections Test

### BenchMark

In [0]:
var bmFilesTestProjection = new BenchmarkMetadata[]{
    // 2021 Q1
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__BE_Projection_DTP.csv"   , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__RA_Projection_DTP.csv"   , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__C_Projection_DTP.csv"    , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__L_Projection_DTP.csv"    , "CH", 2021, 3)
};

### Test

In [0]:
ProjectionCount = 5;
var activity1 = await Import.FromFile("../Files/TransactionalData/NominalCashflows_CH_2020_12_Projection.csv")
            .WithFormat(ImportFormats.Cashflow)
            .WithTarget(DataSource)
            .WithActivityLog()
            .ExecuteAsync();
ProjectionCount = 11;
var activity2 = await Import.FromFile("../Files/TransactionalData/NominalCashflows_CH_2021_3_Projection.csv")
            .WithFormat(ImportFormats.Cashflow)
            .WithTarget(DataSource)
            .WithActivityLog()
            .ExecuteAsync();
ProjectionCount = 1;
(Activity1: activity1.Status, Errors1:activity1.Errors, Activity2:activity2.Status, Errors2:activity2.Errors)

In [0]:
excludedDataNodes = null;
includedDataNodes = new string[]{"DTP1.1"};
var projectionErrorList = await CheckIfrsVariablesValuesAsync(bmFilesTestProjection, excludedDataNodes, includedDataNodes);

In [0]:
projectionErrorList.Count().Should().Be(0);